In [26]:
using LinearAlgebra
using CSV, DataFrames

# N spins
N = 8
Dim = 2^N
# couplings
J_X = 1
J_Y = 1

delta = -2:16

J_Z = 0
B = 0
#decay
alpha = 10

#function for the distance 1/(r_ij ^alpha)
function r_distance(index_i, index_j, alpha)
    dist = abs(index_i - index_j)^alpha
    return (1 / dist)
end

cutoff = 1e-3
# initialise Hamiltonian
Hamiltonian = zeros(Float32, Dim, Dim)

for Ket = 0:Dim-1
    Diagonal = 0.0

    for i = 0:N-1 #we're doing i!=j so here looping first for i<j (effectively half a sum). Then we multiply by 2 every Hamiltonian contribution to account for all pairs. 
        for j = i+1:N-1

            var_ij = r_distance(i, j, alpha)

            #if cutoff not wanted, comment the following lines

            if var_ij < cutoff
                continue
            end

            #S^Z part, similar to before 
            Spin1 = 2 * ((Ket >> i) & 1) - 1
            Spin2 = 2 * ((Ket >> j) & 1) - 1
            Diagonal += -1.0 * r_distance(i, j, alpha) * J_Z * 0.25 * Spin1 * Spin2


            #S^X part 
            bit_i = 2^i
            bit_j = 2^j
            Bra = Ket ⊻ bit_i ⊻ bit_j
            Hamiltonian[Bra+1, Ket+1] += -1.0 * J_X * 0.25 * r_distance(i, j, alpha)

            si = (Ket >> i) & 1
            sj = (Ket >> j) & 1

            #S^Y part- not all terms have the same sign from the fact that S^Y=1/2i (S^+-S^-). I expanded S^Y*S^Y to check which terms would have an overall minus sign. 

            Bra = Ket ⊻ (bit_i) ⊻ (bit_j)
            sign = (si == sj) ? 1 : -1
            Hamiltonian[Bra+1, Ket+1] += -(-0.25) * 1 * J_Y * r_distance(i, j, alpha) * sign


        end

    end

    Hamiltonian[Ket+1, Ket+1] = Diagonal
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1, Ket+1] += -0.5 * B
    end

end



In [28]:

Diag = eigen(Hamiltonian)
EigenVectors = Diag.vectors  #this gives the groundstate eigenvector
EigenEnergies = Diag.values

index = findmin(EigenEnergies)
println(index[2])


#groundstate magnetization per site 
#magnetization along Z
mag_Z_all = zeros(Float32, Dim)
mag_X_all = zeros(Float32, Dim)



for ev in 1:Dim

    mag_Z = 0.0
    for Ket = 0:Dim-1
        spin_sum = 0.0
        for i = 0:N-1
            spin_sum += (2 * ((Ket >> i) & 1) - 1) * 0.5
        end
        mag_Z += spin_sum * abs2(EigenVectors[Ket+1, ev])
    end

    #println("Magnetization along Z per site: ", mag_Z / N)

    #magnetization along X
    mag_X = 0.0

    for Ket = 0:Dim-1
        amp_ket = EigenVectors[Ket+1, ev]
        for i = 0:N-1

            Bra = Ket ⊻ (2^i)
            amp_bra = EigenVectors[Bra+1, ev]
            mag_X += 0.5 * conj(amp_ket) * amp_bra
        end
    end

    #println("Magnetization along X per site: ", mag_X / N)

    mag_Z_all[ev] = mag_Z / N
    mag_X_all[ev] = real(mag_X) / N
end

println(mag_Z_all)





1
Float32[5.1754255f-14, -0.12499998, 0.12499999, 1.0497028f-13, 0.12499995, -0.125, 7.91315f-7, -5.9329018f-8, 0.24999918, -0.24999999, -0.12474686, 0.12474685, 0.12496506, -0.12496499, -0.05980556, 0.24999428, -0.008245879, 0.025694849, -0.24271894, 0.035081215, 9.691001f-13, -0.24238405, -0.036982976, 0.12482119, 0.24981917, -0.005086288, -0.09018696, -0.12499786, 0.1249979, -0.24946842, 0.00013356391, 0.24977078, -0.00043592072, -0.088792615, 0.08879254, 1.1278338f-13, -0.012726346, -0.16193828, -0.0053567016, 0.029166562, 0.008052163, 0.19602533, -0.28144947, 0.03573007, -0.122837685, -0.04947815, 0.37499997, -0.010188571, 6.250756f-13, -0.04804194, -0.081035495, 0.24872494, -0.1338427, -0.058943797, 0.08559262, -0.0805044, -0.07227058, -0.1507239, 0.375, -0.044555552, -0.039399344, -0.10006836, -0.08392159, -0.026836414, 0.009316425, 0.24845825, -0.046948176, -0.12105521, 0.12105523, -0.12694459, -0.12261785, 0.003148049, 0.24641389, -0.0432116, -0.04784441, -0.099959746, -0.2015

In [30]:


#eigen_df = DataFrame(Index=1:length(EigenEnergies), Eigenvalue=EigenEnergies)
#CSV.write("eigenvalues_$(N).csv", eigen_df)


#names = Symbol.("ψ" .* string.(1:Dim))

#eigenvec_df = DataFrame(EigenVectors, names)  # automatically makes Dim columns
#CSV.write("eigenvectors_$(N).csv", eigenvec_df)




In [33]:

ψ_names = Symbol.("ψ" .* string.(1:Dim))
eigenvec_df = DataFrame(EigenVectors, ψ_names)


results_df = DataFrame(
    Index=1:Dim,
    Eigenvalue=EigenEnergies,
    MagZ=mag_Z_all,
    MagX=mag_X_all,
)


combined_df = hcat(results_df, eigenvec_df)


CSV.write("eigenvalues_magnetization_$(N)_XY.csv", results_df)
CSV.write("eigenvectors_$(N)_XY.csv", eigenvec_df)

"eigenvectors_8_XY.csv"

In [ ]:
#XXZ with delta
using LinearAlgebra
using CSV, DataFrames

# N spins
N = 8
Dim = 2^N
# couplings
J_X = 1
J_Y = 1

delta = 1

J_Z = delta
B = 0
#decay
alpha = 10

#function for the distance 1/(r_ij ^alpha)
function r_distance(index_i, index_j, alpha)
    dist = abs(index_i - index_j)^alpha
    return (1 / dist)
end

cutoff = 1e-3
# initialise Hamiltonian
Hamiltonian = zeros(Float32, Dim, Dim)

for Ket = 0:Dim-1
    Diagonal = 0.0

    for i = 0:N-2 #we're doing i!=j so here looping first for i<j (effectively half a sum). Then we multiply by 2 every Hamiltonian contribution to account for all pairs. 
        for j = i+1:N-1

            var_ij = r_distance(i, j, alpha)

            #if cutoff not wanted, comment the following lines

            if var_ij < cutoff
                continue
            end

            #S^Z part, similar to before 
            Spin1 = 2 * ((Ket >> i) & 1) - 1
            Spin2 = 2 * ((Ket >> j) & 1) - 1
            Diagonal += -1.0 * r_distance(i, j, alpha) * J_Z * 0.25 * Spin1 * Spin2


            #S^X part 
            bit_i = 2^i
            bit_j = 2^j
            Bra = Ket ⊻ bit_i ⊻ bit_j
            Hamiltonian[Bra+1, Ket+1] += -1.0 * J_X * 0.25 * r_distance(i, j, alpha)

            si = (Ket >> i) & 1
            sj = (Ket >> j) & 1

            #S^Y part- not all terms have the same sign from the fact that S^Y=1/2i (S^+-S^-). I expanded S^Y*S^Y to check which terms would have an overall minus sign. 

            Bra = Ket ⊻ (bit_i) ⊻ (bit_j)
            sign = (si == sj) ? 1 : -1
            Hamiltonian[Bra+1, Ket+1] += -(-0.25) * 1 * J_Y * r_distance(i, j, alpha) * sign


        end

    end

    Hamiltonian[Ket+1, Ket+1] = Diagonal
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1, Ket+1] += -0.5 * B
    end

end


Diag = eigen(Hamiltonian)
EigenVectors = Diag.vectors  #this gives the groundstate eigenvector
EigenEnergies = Diag.values

index = findmin(EigenEnergies)
println(index[2])


#magnetization per site 
#magnetization along Z
mag_Z_all = zeros(Float32, Dim)
mag_X_all = zeros(Float32, Dim)



for ev in 1:Dim

    mag_Z = 0.0
    for Ket = 0:Dim-1
        spin_sum = 0.0
        for i = 0:N-1
            spin_sum += (2 * ((Ket >> i) & 1) - 1) * 0.5
        end
        mag_Z += spin_sum * abs2(EigenVectors[Ket+1, ev])
    end

    #println("Magnetization along Z per site: ", mag_Z / N)

    #magnetization along X
    mag_X = 0.0

    for Ket = 0:Dim-1
        amp_ket = EigenVectors[Ket+1, ev]
        for i = 0:N-1

            Bra = Ket ⊻ (2^i)
            amp_bra = EigenVectors[Bra+1, ev]
            mag_X += 0.5 * conj(amp_ket) * amp_bra
        end
    end

    #println("Magnetization along X per site: ", mag_X / N)

    mag_Z_all[ev] = mag_Z / N
    mag_X_all[ev] = real(mag_X) / N
end



ψ_names = Symbol.("ψ" .* string.(1:Dim))
eigenvec_df = DataFrame(EigenVectors, ψ_names)


results_df = DataFrame(
    Index=1:Dim,
    Eigenvalue=EigenEnergies,
    MagZ=mag_Z_all,
    MagX=mag_X_all,
)


combined_df = hcat(results_df, eigenvec_df)


CSV.write("eigenvalues_magnetization_$(N)_Heisenberg.csv", results_df)
CSV.write("eigenvectors_$(N)_Heisenberg.csv", eigenvec_df)

1


"eigenvectors_8_Heisenberg.csv"